In [10]:
import pandas as pd
from sqlalchemy import create_engine

In [11]:
database = "extenso_customer_profile"
username = "root"
password = "root"
host = "localhost"
port = "3306"

In [12]:
db_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"


In [13]:
engine = create_engine(db_uri)

In [14]:
tables = pd.read_sql_table("product_category_map", engine)


In [15]:
from vanna.remote import VannaDefault

In [16]:
api_key = "1cdce4ff500d4205b6045ab5f2169a46"
vanna_model_name = "customer_profile1"

In [17]:
vn = VannaDefault(model = vanna_model_name,api_key = api_key)

In [18]:
vn.connect_to_mysql(host = host,dbname = database,user=username,password=password,port=3306)

In [86]:
schemas = vn.run_sql("select * from information_schema.columns where TABLE_SCHEMA = 'extenso_customer_profile'")

In [87]:
schemas

In [88]:
plan = vn.get_training_plan_generic(schemas)
plan

In [89]:
vn.train(plan = plan)

In [90]:
vn.train(documentation='''Here is a table called rw_transaction_data which contains details of each transaction performed by users. The key aspects of this table are as follows:
a. Each row represents a transaction performed by a user.
b. Transactions are associated with specific products, which can include top-up services, airlines, fund transfers, and others.
c. Each transaction has a payer_account_id and a receiver_account_id.
d. Products are uniquely identified using a combination of module_id, product_id, and product_type_id.
e. Transaction details such as transaction amount, cashback amount, reward points, and revenue amount are provided.
f. The sample table includes transactions over a period of time for certain users.''')

In [91]:
vn.train(documentation='''
Here is a table called Product Category Map Table which provides the mapping of products to their parent categories. The key aspects of this table are as follows:
a. It provides the mapping for each product to its parent category.
b. The product_category_id in this table corresponds to the ID in the product category table.
c. The txn_flow column indicates the transaction flow type, which can be:
i. Inflow: For products that credit a user account (e.g., another user’s account is credited).
ii. Outflow: For products that debit a user account (e.g., another user’s account is debited).
iii. Value Chain: For transactions that generate value for the client (e.g., top-up services, airline transactions).
''')

In [92]:
vn.train(documentation=''' for question for each account id give me inflow,outflow and valuechain count
SELECT payer_account_id as account_id,
       SUM(CASE WHEN txn_flow = 'Inflow' THEN 1 ELSE 0 END) AS inflow_count,
       SUM(CASE WHEN txn_flow = 'Outflow' THEN 1 ELSE 0 END) AS outflow_count,
       SUM(CASE WHEN txn_flow = 'Value Chain' THEN 1 ELSE 0 END) AS valuechain_count
FROM rw_transaction_data
JOIN product_category_map ON rw_transaction_data.module_id = product_category_map.module_id
                         AND rw_transaction_data.product_id = product_category_map.product_id
                         AND rw_transaction_data.product_type_id = product_category_map.product_type_id
GROUP BY payer_account_id;
''')

In [93]:
# vn.train(documentation = '''dont use customer profile table for calculation only use it as a reference. For all calculation use rw_transacation_data and product_category_map''')

In [19]:
from vanna.flask import VannaFlaskApp
VannaFlaskApp(vn,  allow_llm_to_see_data=True).run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
None
None
None
None
None
None
None
None
{'function_name': 'get_total_cash_back', 'description': 'What is the total cash back amount for transactions with a cash back amount greater than {cash_back_threshold}?', 'arguments': [{'name': 'cash_back_threshold', 'description': 'The threshold amount for cash back to be considered in the total', 'general_type': 'STRING', 'is_user_editable': True}], 'sql_template': 'SELECT SUM(cash_back_amount) FROM rw_transaction_data WHERE cash_back_amount > {cash_back_threshold};', 'post_processing_code_template': ''}
[]
None
None
None
